In [2]:
import os
from ensemble_boxes import *
import torch
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchPredict,DetBenchTrain
from effdet.efficientdet import HeadNet

from pycocotools.coco import COCO
from tqdm import tqdm

In [3]:
def get_valid_transforms():
    return A.Compose([
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [4]:
class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir, transforms=None):
        super().__init__()
        self.data_dir = data_dir
        # coco annotation 불러오기 (coco API)
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        
        image_id = self.coco.getImgIds(imgIds=index)

        image_info = self.coco.loadImgs(image_id)[0]
        
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        boxes = np.array([x['bbox'] for x in anns])

        # boxex (x_min, y_min, x_max, y_max)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]
        
        labels = np.array([x['category_id'] for x in anns])
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        areas = np.array([x['area'] for x in anns])
        areas = torch.as_tensor(areas, dtype=torch.float32)
                                
        is_crowds = np.array([x['iscrowd'] for x in anns])
        is_crowds = torch.as_tensor(is_crowds, dtype=torch.int64)
                                
        segmentation = np.array([x['segmentation'] for x in anns], dtype=object)

        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([index]), 'area': areas,
                  'iscrowd': is_crowds}

        # transform
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            target['boxes'] = torch.tensor(sample['bboxes'], dtype=torch.float32)

        return image, target, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [5]:
data_dir = './input/data'
annotation = './input/data/test.json'
test_dataset = CustomDataset(annotation, data_dir, get_valid_transforms())

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [6]:
def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [7]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    config.norm_kwargs=dict(eps=.001, momentum=.01)
    
    net = EfficientDet(config, pretrained_backbone=False)    
    
    net.reset_head(num_classes=11)
    net.class_net = HeadNet(config, num_outputs=config.num_classes)    
    
    
    checkpoint = torch.load(checkpoint_path)    
#     net.load_state_dict(checkpoint,strict=False)
    net.load_state_dict(checkpoint['model_state_dict'])
    gc.collect()
    net = DetBenchPredict(net)
    net.eval();
    return net.cuda()

net = load_net('./tf_efficientdet_d5/last-checkpoint.bin')

In [15]:
for images, targets, image_ids in tqdm(data_loader):
    break
# gpu 계산을 위해 image.to(device)
images = torch.stack(images).cuda().float()        
det = net(images)

  0%|          | 0/837 [00:00<?, ?it/s]


In [34]:
def valid_fn(val_data_loader, model, device):
    outputs = []
    for images, targets, image_ids in tqdm(val_data_loader):
        # gpu 계산을 위해 image.to(device)
        images = torch.stack(images).cuda().float()
        output = model(images)
        for out in output:
            outputs.append({'boxes': out['boxes'].tolist(), 'scores': out['scores'].tolist(), 'labels': out['labels'].tolist()})
    return outputs

In [36]:
def valid_fn(val_data_loader, model, device,score_threshold=0.0):
    predictions = []
    for images, targets, image_ids in tqdm(val_data_loader):
        # gpu 계산을 위해 image.to(device)
        images = torch.stack(images).cuda().float()        
        with torch.no_grad():
            det = net(images) #, torch.tensor([1]*images.shape[0]).float().cuda()        
            for i in range(images.shape[0]):
                boxes = det[i].detach().cpu().numpy()[:,:4]    
                scores = det[i].detach().cpu().numpy()[:,4]
                labels = det[i].detach().cpu().numpy()[:,5]
                indexes = np.where(scores > score_threshold)[0]
                boxes = boxes[indexes]
                boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
                boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
                predictions.append({
                    'boxes': boxes[indexes],
                    'scores': scores[indexes],
                    'labels': labels[indexes]
                })
    return predictions

In [37]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
outputs = valid_fn(data_loader, net, device)

 44%|████▍     | 372/837 [00:47<00:57,  8.07it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in add
  
100%|██████████| 837/837 [01:46<00:00,  7.87it/s]


In [38]:
prediction_strings = []
file_names = []
coco = COCO(annotation)
score_threshold = 0.05
for i, output in enumerate(outputs):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
        if score > score_threshold:
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
                box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(f'submission.csv', index=None)
print(submission.head())

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
                                    PredictionString              image_id
0  2.0 0.21496797 1031.3042 238.27869 2114.628 52...  batch_01_vt/0021.jpg
1  2.0 0.21141881 967.3356 243.84966 1985.8867 53...  batch_01_vt/0028.jpg
2  2.0 0.21466209 963.8663 244.38239 1980.0585 53...  batch_01_vt/0031.jpg
3  8.0 0.22500965 1030.694 239.30667 2114.3645 53...  batch_01_vt/0032.jpg
4  2.0 0.199258 1032.443 238.38881 2115.4553 528....  batch_01_vt/0070.jpg
